In [ ]:
import numpy as np
import cv2 as cv
from scipy.optimize import linear_sum_assignment

cost = np.array([
    [0, 1, 2],
    [1, 0, 2],
    [1, 0, 2],
])
cost

In [ ]:
row_ind, col_ind = linear_sum_assignment(cost)
row_ind, col_ind

In [ ]:
row_ind